In [102]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime

In [103]:
def reverse_date(btc, remove_date="Yes"):
    '''Reverses the dataset so it is in chronological order. Optional to remove date column and set as index.'''
    final_data = btc.reindex(index=btc.index[::-1])
    
    #Fixing incorrect dates 
    for i in range(0, len(btc)): 
        if "v" in btc.ix[i,'Date']:
            new_date = btc.ix[i,'Date'].replace('v', '/')
            final_data.ix[i,'Date'] = new_date
            
    if remove_date == "Yes":
        # Need to reverse order of dataframe.
        final_data = final_data.set_index(final_data['Date'])
        del final_data['Date']
    return final_data

In [104]:
btc = pd.read_csv('https://raw.githubusercontent.com/chrishyland/Crypto-position/master/bitcoin_final.csv')
crix = pd.read_csv('https://raw.githubusercontent.com/chrishyland/Crypto-position/master/crix.csv')

df = reverse_date(btc, "No")
df.head()

,Date,Open,High,Low,Close,Volume,MarketCap
1669,28/04/2013,135.30,135.98,132.10,134.21,0,1500520000
1668,29/04/2013,134.44,147.49,134.00,144.54,0,1491160000
1667,30/04/2013,144.00,146.93,134.05,139.00,0,1597780000
1666,01/05/2013,139.00,139.89,107.72,116.99,0,1542820000
1665,02/05/2013,116.38,125.60,92.28,105.21,0,1292190000


In [105]:
def ichimoku_plot(df):
    '''Computes the Ichimoku Kinkō Hyō trend identification system.'''
    # This plot has 5 components to it. 
    high_prices = df['High']
    close_prices = df['Close']
    low_prices = df['Low']
    dates = df.index
    
    # Ichimoku (Conversion Line): (9-period high + 9-period low)/2
    nine_period_high = df['High'].rolling(window=9, center=False).max() # Usually window is 9 days.
    nine_period_low = df['Low'].rolling(window=9, center=False).max() 
    ichimoku = (nine_period_high + nine_period_low) /2
    df['tenkan_sen'] = ichimoku
    
    # Kijun-Sen (Base line): (26-period high + 26-period low)/2)
    period26_high = high_prices.rolling(window=26, center=False).max() # Window normally 26 days.
    period26_low = low_prices.rolling(window=26, center=False).min()
    df['kijun_sen'] = (period26_high + period26_low) / 2
    
    # Senkou Span A (Leading span A): (Base line + Conversion line) / 2
    df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(26) 

    # Senkou Span B (Leading span B): (52-period high + 52-period low)/2
    period52_high = high_prices.rolling(window=52, center=False).max()
    period52_low = low_prices.rolling(window=52, center=False).min()
    df['senkou_span_b'] = ((period52_high + period52_low) / 2).shift(26)
    
    # Chikou Span (Lagging span): Closing price of last 22 periods
    df['chikou_span'] = close_prices.shift(-22)
    
    return df[df.columns[6:]]

In [106]:
def sma_plot(df, window):
    '''Computes simple moving average.'''
    rolling = df['Close'].rolling(window=window) # Window tells us how many days average to take.
    return rolling.mean()

In [107]:
def bollinger_plot(df, window, num_sd):
    '''Computes Bollinger bands depending on number of standard deviation and window.''' 
    rolling_mean = df['Close'].rolling(window).mean() # Window should be same as SMA.
    rolling_std = df['Close'].rolling(window).std()
    
    bollinger = pd.DataFrame(data=None)
    bollinger['Rolling Mean'] = rolling_mean
    bollinger['Bollinger High'] = rolling_mean + (rolling_std * num_sd)
    bollinger['Bollinger Low'] = rolling_mean - (rolling_std * num_sd)

    return bollinger

In [108]:
# TODO: Configure Bokeh to allow for toggle configuration for sliding bar. Allows us to choose the date.
# Example: - https://finance.yahoo.com/chart/AAPL#eyJpbnRlcnZhbCI6ImRheSIsInBlcmlvZGljaXR5IjoxLCJ0aW1lVW5pdCI6bnVsbCwiY2FuZGxlV2lkdGgiOjgsInZvbHVtZVVuZGVybGF5Ijp0cnVlLCJhZGoiOnRydWUsImNyb3NzaGFpciI6dHJ1ZSwiY2hhcnRUeXBlIjoibGluZSIsImV4dGVuZGVkIjpmYWxzZSwibWFya2V0U2Vzc2lvbnMiOnt9LCJhZ2dyZWdhdGlvblR5cGUiOiJvaGxjIiwiY2hhcnRTY2FsZSI6ImxpbmVhciIsInBhbmVscyI6eyJjaGFydCI6eyJwZXJjZW50IjoxLCJkaXNwbGF5IjoiQUFQTCIsImNoYXJ0TmFtZSI6ImNoYXJ0IiwidG9wIjowfX0sInNldFNwYW4iOnt9LCJsaW5lV2lkdGgiOjIsInN0cmlwZWRCYWNrZ3JvdWQiOnRydWUsImV2ZW50cyI6dHJ1ZSwiY29sb3IiOiIjMDA4MWYyIiwic3ltYm9scyI6W3sic3ltYm9sIjoiQUFQTCIsInN5bWJvbE9iamVjdCI6eyJzeW1ib2wiOiJBQVBMIn0sInBlcmlvZGljaXR5IjoxLCJpbnRlcnZhbCI6ImRheSIsInRpbWVVbml0IjpudWxsLCJzZXRTcGFuIjp7fX1dLCJzdHVkaWVzIjp7InZvbCB1bmRyIjp7InR5cGUiOiJ2b2wgdW5kciIsImlucHV0cyI6eyJpZCI6InZvbCB1bmRyIiwiZGlzcGxheSI6InZvbCB1bmRyIn0sIm91dHB1dHMiOnsiVXAgVm9sdW1lIjoiIzAwYjA2MSIsIkRvd24gVm9sdW1lIjoiI0ZGMzMzQSJ9LCJwYW5lbCI6ImNoYXJ0IiwicGFyYW1ldGVycyI6eyJoZWlnaHRQZXJjZW50YWdlIjowLjI1LCJ3aWR0aEZhY3RvciI6MC40NSwiY2hhcnROYW1lIjoiY2hhcnQifX0sIuKAjEJvbGxpbmdlciBCYW5kc%2BKAjCAoQywxMCwyLG1hLHkpIjp7InR5cGUiOiJCb2xsaW5nZXIgQmFuZHMiLCJpbnB1dHMiOnsiRmllbGQiOiJDbG9zZSIsIlBlcmlvZCI6IjEwIiwiU3RhbmRhcmQgRGV2aWF0aW9ucyI6MiwiTW92aW5nIEF2ZXJhZ2UgVHlwZSI6InNpbXBsZSIsIkNoYW5uZWwgRmlsbCI6dHJ1ZSwiaWQiOiLigIxCb2xsaW5nZXIgQmFuZHPigIwgKEMsMTAsMixtYSx5KSIsImRpc3BsYXkiOiLigIxCb2xsaW5nZXIgQmFuZHPigIwgKEMsMTAsMixtYSx5KSJ9LCJvdXRwdXRzIjp7IkJvbGxpbmdlciBCYW5kcyBUb3AiOiIjZmZkYjQ4IiwiQm9sbGluZ2VyIEJhbmRzIE1lZGlhbiI6IiNmZmEzM2YiLCJCb2xsaW5nZXIgQmFuZHMgQm90dG9tIjoiI2ZmYmQ3NCJ9LCJwYW5lbCI6ImNoYXJ0IiwicGFyYW1ldGVycyI6eyJjaGFydE5hbWUiOiJjaGFydCJ9fX19

In [109]:
'''Constructing candle price chart with ichimoku plot'''
from bokeh.events import ButtonClick
from bokeh.layouts import column, row, widgetbox
from bokeh.models.widgets import DataTable, TableColumn, Button
from bokeh.plotting import figure, output_file, show, ColumnDataSource, curdoc
from bokeh.models import HoverTool, CustomJS, Legend
from math import pi 

df['Date'] = pd.to_datetime(df['Date'], dayfirst = True) 

inc = df.Close > df.Open
dec = df.Open > df.Close
w = 12*60*60*1000 #half day in ms

#Display last 6 months by default 
df_6m = df.iloc[-180:,]['Date']

TOOLS = "pan, wheel_zoom, box_zoom, reset, save, hover"
p = figure(x_axis_type = "datetime", tools = TOOLS,
           plot_width = 1000, title = "Bitcoin Candlestick Chart", x_range = (df_6m.min(), df_6m.max())) 
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha = 0.30 

#Construct increasing and decreasing lines 
p.segment(df.Date[inc], df.High[inc], df.Date[inc], df.Low[inc], color="#17BECF")
p.segment(df.Date[dec], df.High[dec], df.Date[dec], df.Low[dec], color="#FF7777")

#Construct increasing and decreasing bars 
p.vbar(df.Date[inc], w, df.Open[inc], df.Close[inc], fill_color="#17BECF", line_color="#17BECF")
p.vbar(df.Date[dec], w, df.Open[dec], df.Close[dec], fill_color="#FF7777", line_color="#FF7777")

sourceInc=ColumnDataSource(ColumnDataSource.from_df(df.loc[inc]))
sourceDec=ColumnDataSource(ColumnDataSource.from_df(df.loc[dec]))

#Constructing hover tool for candle plot. Currently not working 
#hover = p.select(dict(type=HoverTool))
#hover.mode = "vline"
'''hover.tooltips = [("Date", "@Date{%F}"),
                  ("Open", "@Open{0.2f}"), 
                  ("High", "@High{0.2f}"), 
                  ("Low", "@Low{0.2f}"), 
                  ("Close", "@Close{0.2f}")
                  ("Volume", "@Volume"]'''
#hover.formatters = {'Date' : 'datetime'}

#add line render to display ichimoku plot 
ichimoku_plot(df)
r1 = p.line(df['Date'], df['tenkan_sen'], line_width = 1, color = "#92FFB4")
r2 = p.line(df['Date'], df['kijun_sen'], line_width = 1, color = "#92B2FF")
r3 = p.line(df['Date'], df['senkou_span_a'], line_width = 1, color = "#C092FF")
r4 = p.line(df['Date'], df['senkou_span_b'], line_width = 1, color = "#FFE592")
r5 = p.line(df['Date'], df['chikou_span'], line_width = 1, color = "#6878FB")

#Add bar render to display scaled down volume.Change bar colours upward/downward trend 
p.vbar(df['Date'], w, df['Volume']/3000000, 0, color="#5DE0F6")

'''Fill area between senkou span A and B. If A above B, colour green. If B above A, color red'''
index = 0
index_a = np.argwhere(np.isnan(df['senkou_span_a'].values)).max()
index_b = np.argwhere(np.isnan(df['senkou_span_b'].values)).max()
if index_b > index_a: 
    index = index_b
else:
    index = index_a 
dates = df['Date'].values[index:]
senkou_span_a = df['senkou_span_a'].values[index:]
senkou_span_b = df['senkou_span_b'].values[index:]
band_y = np.append(senkou_span_a, senkou_span_b[::-1])
band_x = np.append(dates, dates[::-1])
p.patch(band_x, band_y, color='#8EF3DA', fill_alpha = 0.20)

#Display legend 
legend = Legend(items = [
    ("Tenkan-Sen", [r1]),
    ("Kijun-Sen", [r2]),
    ("Senkou Span A", [r3]),
    ("Senkou Span B", [r4]),
    ("Chikou Span", [r5])
], location = (0,399))
p.add_layout(legend, 'right')

#Adding button widgets 
button_3m = Button(label= "3 month")
button_6m = Button(label = "6 month")
button_1y = Button(label = "1 Year")
button_ytd = Button(label = "YTD")
button_all = Button(label = "All")

def update_3m(): 
    '''Update zoom to 3 months'''
    df_3m = df.iloc[-90:,]['Date']
    p.x_range.start = df_3m.min()
    p.x_range.end = df_3m.max()

def update_6m():
    '''Update zoom to 6 months'''
    df_6m = df.iloc[-180:,]['Date']
    p.x_range.start = df_6m.min()
    p.x_range.end = df_6m.max()

def update_1y():
    '''Update zoom to 12 months'''
    df_1y = df.iloc[-365:,]['Date']
    p.x_range.start = df_1y.min()
    p.x_range.end = df_1y.max() 

def update_ytd():
    '''Update zoom to start of year'''
    start_date = datetime.strptime("2017-1-01", "%Y-%m-%d")
    p.x_range.start = start_date
    p.x_range.end = df['Date'].max() 

def update_all(): 
    '''Update zoom to display all data'''
    p.x_range.start = df['Date'].min() 
    p.x_range.end = df['Date'].max() 
    
button_3m.on_click(update_3m)
button_6m.on_click(update_6m)
button_1y.on_click(update_1y)
button_ytd.on_click(update_ytd)
button_all.on_click(update_all)

#Add dropbox widget 
button_controls = widgetbox([button_3m, button_6m, button_1y, button_ytd, button_all], width = 100)

#Set layout and show plot. Use curdoc to get widgets working 
output_file("candlestick.html", title="candlestick.py")
layout = row(button_controls, p)
#curdoc().add_root(layout)
show(layout)

In [3]:
# output to static HTML file
output_file("lines.html")

# create a new plot with a title and axis labels
p = figure(title="SMA of bitcoin", x_axis_label='Date', y_axis_label='Price')

# add a line renderer with legend and line thickness
p.line(df.index, df.High, legend="Temp.", line_width=2)

# show the results
show(p)

NameError: name 'df' is not defined

In [72]:
import plotly
plotly.tools.set_credentials_file(username='chrishigh95', api_key='Mg0N9OIjeBB64gnsBP1Y')
import plotly.plotly as py
from plotly.graph_objs import *

In [10]:
data = [Bar(x=df.index,
            y=sma_plot(df, 9))]

py.iplot(data)

In [73]:
import numpy as np

import plotly.plotly as py

# Create a trace
trace = go.Ohlc(
    open = dt['Open'],
    high = dt['High'],
    low = dt['Low'],
    close = dt['Close']
)
data = [trace]

layout = dict(
    title = "BTC USD Cross"
)
fig = dict(data=data, layout=layout)

py.iplot(fig, filename='basic-line')

NameError: name 'go' is not defined

In [74]:
boll_low = go.Scatter(
    x=dt.index,
    y=dt['Bollinger High'],
    name = "Bollinger High",
    line = dict(color = '#66c2ff'),
    opacity = 0.8)

boll_high = go.Scatter(
    x=dt.index,
    y=dt['Bollinger Low'],
    name = "Bollinger Low",
    line = dict(color = '#99d6ff'),
    opacity = 0.8)
close = go.Scatter(
    x=dt.index,
    y=dt['Close'],
    name = "Close",
    line = dict(color = '#006bb3')

)

data1 = [boll_low, boll_high, close]
layout1 = dict(
    title='Bollinger bands',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(range="28/4/2013 - 22/11/2017"),
        type='date'
    )
)

fig = dict(data=data1, layout=layout1)

py.iplot(fig, filename='basic-line2')

NameError: name 'go' is not defined